### Importações

In [1]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [2]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [3]:
today = date.today()
tomorrow = today + timedelta(days=1)

day = tomorrow

In [4]:
data_var = day.strftime('%Y-%m-%d')
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
# ligas = temporada[temporada['Date'] == data_var]['League'].unique().tolist()
# temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos.to_csv(f'jogos_processados/{data_var}.csv', index=False)

jogos[['Date', 'League', 'Time', 'Home', 'Away', 
       'Odds_H', 'Odds_A', 'Over_Line', 'Odds_Over', 'Odds_Under', 'HA_Line',
       'HA_Odds_H', 'HA_Odds_A', ]]

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A
6780,2023-11-16,Brasil Nbb,19:00,Flamengo,Brasília,1.03,11.50,160.5,1.86,1.86,-19.5,2.07,1.67
6781,2023-11-16,Brasil Nbb,19:00,Minas,Bauru,1.32,3.30,154.5,1.81,1.85,-8.5,1.96,1.75
6782,2023-11-16,Brasil Nbb,19:30,Pato,Franca,6.25,1.11,161.5,1.87,1.89,12.5,1.95,1.76
6783,2023-11-16,Eua Nba,00:00,Los Angeles Lakers,Sacramento Kings,1.97,1.97,234.5,1.86,2.07,-1.5,2.07,1.88
6784,2023-11-16,Eua Nba,00:00,Portland Trail Blazers,Cleveland Cavaliers,4.73,1.24,217.5,1.86,2.07,9.5,2.05,1.89
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6855,2023-11-16,Suécia Liga De Basquete,15:00,Hogsbo F,Sjuharads F,1.25,3.64,133.5,1.82,1.84,-8.5,1.96,1.75
6856,2023-11-16,Turquia Kbsl Feminina,08:00,Antalya 07 F,Galatasaray F,4.41,1.19,161.5,1.88,1.79,10.5,1.99,1.73
6857,2023-11-16,Turquia Kbsl Feminina,08:00,Tarsus F,Izmit Bld. F,3.70,1.25,152.5,1.84,1.84,7.5,2.08,1.67
6858,2023-11-16,Turquia Kbsl Feminina,12:00,Bellona Kayseri F,CBK Mersin F,4.51,1.18,159.5,1.87,1.79,9.5,2.09,1.66


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
3,00:00,Eua Nba,Los Angeles Lakers,Sacramento Kings,234.5,1.86,1.0000,Over
4,00:00,Eua Nba,Portland Trail Blazers,Cleveland Cavaliers,217.5,1.86,1.0000,Over
5,21:30,Eua Nba,Miami Heat,Brooklyn Nets,216.5,1.87,1.0000,Over
70,23:30,México Lnbp,Panteras,Libertadores,168.5,1.87,0.7983,Over


### Over v2 (NB)

In [9]:
# Duplicando o dataset
df_over_v2 = jogos.copy()

df_over_v2.reset_index(drop=True, inplace=True)
df_over_v2.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v2.empty:
    mdl_over_v2 = load_model('ML/over/v2/v2_over', verbose=False)
    prev = predict_model(mdl_over_v2, df_over_v2)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo
    prev = prev[((prev['Last_CG_H'] >= 0) & (prev['Last_CG_H'] < 130)) | ((prev['Last_CG_H'] >= 183) & (prev['Last_CG_H'] < 1485))]

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v2_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V2 (NB) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V2 (NB) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
4,00:00,Eua Nba,Portland Trail Blazers,Cleveland Cavaliers,217.5,1.86,0.5713,Over
5,21:30,Eua Nba,Miami Heat,Brooklyn Nets,216.5,1.87,0.9988,Over
6,16:05,Europa Euroliga,Maccabi Tel Aviv,Fenerbahce,163.5,1.91,0.7198,Over
7,16:15,Europa Euroliga,Olympiakos,Crvena zvezda,153.5,1.91,0.7131,Over
8,16:30,Europa Euroliga,Milano,Anadolu Efes,156.5,1.82,0.6493,Over
9,16:45,Europa Euroliga,Real Madrid,Monaco,165.5,1.91,0.7242,Over
13,08:35,China Cba,Ningbo Rockets,Guangzhou,198.5,1.87,0.8300,Over
14,08:35,China Cba,Shandong,Nanjing Tongxi,200.5,1.80,0.9403,Over
30,00:00,Eua Ncaa,Nevada,Pacific,151.5,1.91,0.9893,Over
31,00:00,Eua Ncaa,Seattle,Northern Arizona,141.5,1.91,1.0000,Over


### Over v3 (NB)

In [10]:
# Duplicando o dataset
df_over_v3 = jogos.copy()

df_over_v3.reset_index(drop=True, inplace=True)
df_over_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v3.empty:
    mdl_over_v3 = load_model('ML/over/v3/v3_over', verbose=False)
    prev = predict_model(mdl_over_v3, df_over_v3)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v3_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V3 (NB) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V3 (NB) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
2,19:30,Brasil Nbb,Pato,Franca,161.5,1.87,0.5191,Over
3,00:00,Eua Nba,Los Angeles Lakers,Sacramento Kings,234.5,1.86,0.7336,Over
4,00:00,Eua Nba,Portland Trail Blazers,Cleveland Cavaliers,217.5,1.86,0.7286,Over
5,21:30,Eua Nba,Miami Heat,Brooklyn Nets,216.5,1.87,0.7764,Over
6,16:05,Europa Euroliga,Maccabi Tel Aviv,Fenerbahce,163.5,1.91,0.7968,Over
7,16:15,Europa Euroliga,Olympiakos,Crvena zvezda,153.5,1.91,0.7955,Over
9,16:45,Europa Euroliga,Real Madrid,Monaco,165.5,1.91,0.7911,Over
11,07:30,Austrália Nbl,Perth Wildcats,Cairns Taipans,175.5,1.88,0.5334,Over
30,00:00,Eua Ncaa,Nevada,Pacific,151.5,1.91,0.8145,Over
31,00:00,Eua Ncaa,Seattle,Northern Arizona,141.5,1.91,0.8145,Over
